In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=07b96aa5d6464cfd6260f52f0fcca87e7cb0b491519447857fb27947649a32c9
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [ ]:
import numpy as np
import nltk, string
import dill
import pickle
import torch
import tflearn
import json
import os
import re
from sklearn.metrics.pairwise import cosine_similarity


nltk.download('punkt') # if necessary...


Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
PATH = os.path.dirname(os.path.realpath('__file__'))

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

list_words = []
labels = []
docs_x = [] 
docs_y = [] 

def cleanText(Text):
    cleanedText = Text.replace('\n',' ').lower()
    cleanedText = cleanedText.replace("\xc2\xa0", "")
    cleanedText = re.sub('([,])','', cleanedText)
    cleanedText = re.sub(' +',' ', cleanedText)
    cleanedText = cleanedText.encode('ascii', 'ignore').decode('ascii')
    return cleanedText

def getData():
    listTexts = source_data['abstract'].tolist()
    finallist=[]
    print(listTexts)
    for i in range (0,4):
        textDictionary = {"tag":i}
        listTexts_i = cleanText(listTexts[i])
        print(listTexts_i)
        textDictionary.update(texts = listTexts_i)
        finallist.append(textDictionary)
    finalDictionary={"intents":finallist}       
    return finalDictionary


combinedDictionary = dict()
print ('Getting Text Data')
combinedDictionary.update(getData())
print ('Total len of dictionary', len(combinedDictionary))

print ('Saving text data dictionary')
np.save(PATH + '/textDictionary.npy', combinedDictionary)

with open(PATH + '/file.txt', 'w') as file:
     file.write(json.dumps(combinedDictionary))



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
with open(PATH + "/file.txt") as file:
    dataset = json.load(file)
    
for intent in dataset["intents"]:
    for pattern in intent["texts"]: 
        split_words = nltk.word_tokenize(pattern) 
        list_words.extend(split_words) 
        docs_x.append(split_words) 
        docs_y.append(intent["tag"]) 
    if intent["tag"] not in labels:
        labels.append(intent["tag"]) 

list_words = [stemmer.stem(w.lower()) for w in list_words if w != "?"] 
list_words = sorted(list(set(list_words))) 

labels = sorted(labels)

training = [] 
output = [] 

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = [] 

    split_words = [stemmer.stem(w.lower()) for w in doc] 

    for w in list_words:
        if w in split_words:
            bag.append(1) 
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1 

    training.append(bag) 
    output.append(output_row) 

In [ ]:
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))


def bag_of_words(s, list_words): 
    bag = [0 for _ in range(len(list_words))]

    inp_str_words = nltk.word_tokenize(s)
    inp_str_words = [stemmer.stem(word.lower()) for word in inp_str_words]

    for search_element in inp_str_words:
        for i, w in enumerate(list_words):
            if w == search_element:
                bag[i] = 1 
            
    return np.array(bag)

In [ ]:
# Transformer Model
training = np.array(training) 
output = np.array(output)

net = tflearn.input_data(shape=[None, len(training[0])]) 
net = tflearn.fully_connected(net, 8) #Hidden layer with 8 neurons.
net = tflearn.fully_connected(net, 8) #Another hidden layer with 8 neurons.
net = tflearn.fully_connected(net, 8) #Another hidden layer with 8 neurons.
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") 

model = tflearn.DNN(net) 
try:
    model.predict(PATH + "/model.tflearn")
    model.load()
except:
    model.fit(training, output, n_epoch=2000, batch_size=8, show_metric=True)
    model.save(PATH + "/model.tflearn") 

Training Step: 7437  | total loss: 1.30984 | time: 0.929s
| Adam | epoch: 014 | loss: 1.30984 - acc: 0.2777 -- iter: 1984/4423


In [ ]:
def get_cosine_similarity(text1, text2):
    tfidf = TfidfVectorizer().fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

    
def is_similar(similarity_score, similarity_threshold):

  is_similar = False

  if(similarity_score >= similarity_threshold):
    is_similar = True

  return is_similar

def run_similarity_analysis(query_text, similarity_threshold=0.8):

    top_N=10

    # Preprocess the given document
    preprocessed_document = preprocess_text(query_text)
    
    # Convert the given document into numerical features
    X_new = tfidf.transform([preprocessed_document])
    
    # Compute cosine similarity between the given document and each research paper
    similarities = cosine_similarity(training_data, X_new)

    # Sort the research papers based on their similarity with the given document
    indices = np.argsort(similarities, axis=0)
    indices = indices[::-1]
    
    # Get the top N research papers with the highest similarity
    top_n_indices = indices[:top_N, 0]
    
    # Compute the similarity percentage with the top 10 research papers
    similarity_percentage = similarities[top_n_indices, 0] * 100

    # assign data of lists.
    texts = source_data['abstract'].tolist();
    similar_articles = {'abstract': [texts[i] for i in top_n_indices], 'similarity': similarity_percentage}  
  
    # Create DataFrame  
    formated_result = pd.DataFrame(similar_articles)
    
    # Create JSON Array
    similarity_decision=[]
    for x in formated_result.iloc:
        json={'similarity_score': x["similarity"], 
                           'similarity_percentage': str(round(x["similarity"])) + '%',
                            'similar_article': x["abstract"]
                        }
        similarity_decision.append(json)
    return similarity_decision

In [ ]:
new_incoming_text = "hi, hello, how are you?"

# Run the plagiarism detection
analysis_result = run_similarity_analysis(new_incoming_text, similarity_threshold=0.8)
analysis_result

[{'similarity_score': 24.753818061915094,
  'similarity_percentage': '25%',
  'similar_article': "How do you persuade philanthropists to pay $1 million for every pathogenic human virus you discover? Anjali Nayar talks to 'virus hunter' Nathan Wolfe in Cameroon to find out."},
 {'similarity_score': 21.506302762940383,
  'similarity_percentage': '22%',
  'similar_article': 'gas velocity dispersion measures a amount of disordered motions of the rotating disk. accurate estimates of this parameter are of a utmost importance because it was directly linked to disk stability and star formation. the global measure of a gas velocity dispersion should be inferred from a width of a atomic hydrogen hi 21 cm line. we explore how several systematic effects involved inside a production of hi cubes affect a approximate of hi velocity dispersion. we do so by comparing a hi velocity dispersion derived from different types of data cubes provided by a hi nearby galaxy survey (things). we find that residual

In [ ]:
new_incoming_text =  source_data.iloc[1]['abstract']

# Run the plagiarism detection
analysis_result = run_plagiarism_analysis(new_incoming_text, similarity_threshold=0.8)
analysis_result

[{'similarity_score': 100.00000000000004,
  'similarity_percentage': '100%',
  'similar_article': 'A critical aspect of reliable communication involves the design of codes that\nallow transmissions to be robustly and computationally efficiently decoded\nunder noisy conditions. Advances in the design of reliable codes have been\ndriven by coding theory and have been sporadic. Recently, it is shown that\nchannel codes that are comparable to modern codes can be learned solely via\ndeep learning. In particular, Turbo Autoencoder (TURBOAE), introduced by Jiang\net al., is shown to achieve the reliability of Turbo codes for Additive White\nGaussian Noise channels. In this paper, we focus on applying the idea of\nTURBOAE to various practical channels, such as fading channels and chirp noise\nchannels. We introduce TURBOAE-TI, a novel neural architecture that combines\nTURBOAE with a trainable interleaver design. We develop a carefully-designed\ntraining procedure and a novel interleaver penal

In [ ]:
pickle.dump(run_plagiarism_analysis, open('similarity_model.pkl','wb'))
g = pickle.load(open('similarity_model.pkl','rb'))
g("possible further increases in nucleic acid", similarity_threshold=0.8)

[{'similarity_score': 55.90983656174052,
  'similarity_percentage': '56%',
  'similar_article': 'Nucleic acid-based therapeutics present huge potential in the treatment of pulmonary diseases ranging from lung cancer to asthma and chronic pulmonary diseases, which are often fatal and widely prevalent. The susceptibility of nucleic acids to degradation and the complex structure of lungs retard the effective pulmonary delivery of nucleic acid drug. To overcome these barriers, different strategies have been exploited to increase the delivery efficiency using chemically synthesized nucleic acids, vector encapsulation, proper formulation, and administration route. However, several limitations regarding off-target effects and immune stimulation of nucleic acid drugs hamper their translation into the clinical practice. Therefore, their successful clinical application will ultimately rely on well-developed carriers and methods to ensure safety and efficacy. In this review, we provide a comprehe